
********************************************************************** 
  Note: This license has also been called the "New BSD License" or 
  "Modified BSD License". See also the 2-clause BSD License.
 
  Copyright © 2018-2019 - General Electric Company, All Rights Reserved
  
  Project: ANSWER, developed with the support of the Defense Advanced 
  Research Projects Agency (DARPA) under Agreement  No.  HR00111990006. 
 
  Redistribution and use in source and binary forms, with or without 
  modification, are permitted provided that the following conditions are met:
  1. Redistributions of source code must retain the above copyright notice, 
     this list of conditions and the following disclaimer.
 
  2. Redistributions in binary form must reproduce the above copyright notice, 
     this list of conditions and the following disclaimer in the documentation 
     and/or other materials provided with the distribution.
 
  3. Neither the name of the copyright holder nor the names of its 
     contributors may be used to endorse or promote products derived 
     from this software without specific prior written permission.
 
  THIS SOFTWARE IS PROVIDED BY THE COPYRIGHT HOLDERS AND CONTRIBUTORS "AS IS" 
  AND ANY EXPRESS OR IMPLIED WARRANTIES, INCLUDING, BUT NOT LIMITED TO, THE 
  IMPLIED WARRANTIES OF MERCHANTABILITY AND FITNESS FOR A PARTICULAR PURPOSE 
  ARE DISCLAIMED. IN NO EVENT SHALL THE COPYRIGHT HOLDER OR CONTRIBUTORS BE 
  LIABLE FOR ANY DIRECT, INDIRECT, INCIDENTAL, SPECIAL, EXEMPLARY, OR 
  CONSEQUENTIAL DAMAGES (INCLUDING, BUT NOT LIMITED TO, PROCUREMENT OF 
  SUBSTITUTE GOODS OR SERVICES; LOSS OF USE, DATA, OR PROFITS; OR BUSINESS 
  INTERRUPTION) HOWEVER CAUSED AND ON ANY THEORY OF LIABILITY, WHETHER IN 
  CONTRACT, STRICT LIABILITY, OR TORT (INCLUDING NEGLIGENCE OR OTHERWISE) 
  ARISING IN ANY WAY OUT OF THE USE OF THIS SOFTWARE, EVEN IF ADVISED OF 
  THE POSSIBILITY OF SUCH DAMAGE.

 ***********************************************************************


# Demonstration and Usage Documentation of K-CHAIN Services

This notebook demonstrates the different functionality and capabilities of the Knowledege-consistent hybrid AI networks (K-CHAIN). Specifically, the demonstration will cover the following aspects:

-  Setup of K-CHAIN
-  Building models:
    -  with physics models with simple equations provided as strings
    -  with physics models captured in TF-compatible python code, where the code itself was derived from extracted text
    -  with default values of certain inputs 
-  Evaluating models during inference:
    -  agnostic to whether model was built as data-driven or physics-based
    -  with ability to use default values (if not provided at inference time) and inform user about all default values used in the computation 
    -  with ability to inform user that key variables are missing to conduct inference
- Appending existing models with new model fragments, where:
    -  new model fragment uses outputs from existing model
    -  new model fragment is input into existing model
    -  new model fragment needs gradient information of nodes in existing model


# Setup of Services

In [2]:
#imports needed for demonstration

#for communicating with services
import requests

This code demonstrates the use of K-CHAIN service. Please use "Launch K-CHAIN Service" Notebook for launching this service before proceeding to the following demonstrations. The code below assumes that service has been launched. 

The "Launch K-CHAIN Service" Notebook is available [here](Launch%20K-CHAIN%20Service.ipynb).

In [3]:
#URL to interact with build
url_build = 'http://localhost:12345/darpa/aske/kchain/build'

#URL to interact with evaluate service
url_append = 'http://localhost:12345/darpa/aske/kchain/append'

#URL to interact with evaluate service
url_evaluate = 'http://localhost:12345/darpa/aske/kchain/evaluate'

## Model build demonstrations

### Build physics models with simple equations provided as strings:

In [3]:
#inputPacket like this one is programmatically constructed by the ANSWER agent
inputPacket = {
                  "inputVariables": [
                    {
                      "name": "Mass",
                      "type": "double"
                    },
                    {
                      "name": "Acceleration",
                      "type": "double"
                    }
                  ],
                  "outputVariables": [
                    {
                      "name": "Force",
                      "type": "double"
                    }
                  ],
                   "equationModel" : "Force = Mass * Acceleration",
                   "modelName" : "Newtons2LawModel"
                 }

#send request to build model
r = requests.post(url_build, json=inputPacket)

#see the response
r.json()

{'metagraphLocation': '../models/Newtons2LawModel',
 'modelType': 'Physics',
 'trainedState': 0}

__Explanation of ideal outcome:__ 

```
{'metagraphLocation': '../models/Newtons2LawModel',
 'modelType': 'Physics',
 'trainedState': 0}
```

TensorFlow models are locally stored as a MetaGraph which includes the computational graph object, model parameters, and data associated with training that model (see [TensorFlow documentation](https://www.tensorflow.org/guide/saved_model#save_and_restore_models)). This model is saved in a folder called "models". In this case, the model was of type physics. Demo 3 below shows a case with data-driven model. This model has not been trained as there are no parameters. This service does not yet support physics model with trainable parameters, it will be available in a future release.  

The computational graph capturing this _Newtons2LawModel_ can be seen in __TensorBoard__. The model is as follows:
    <img src="figures for notebook/n2l_tensorboard_pic.PNG" style="width: 80%">

Note:
One can open TensorBoard by typing the following in cmd prompt from _kchain_ folder:
```
tensorboard --logdir="log/example/model"
```
The resulting computational graph is available under Graph tab of TensorBoard or by going to http://localhost:6006/#graphs in your browser after running TensorBoard.

### Build physics models captured in TF-compatible python code, where the code itself was derived from extracted text :

The text2triples service extracts concepts and equations from text/HTML documents, such as this [Speed of Sound page](https://www.grc.nasa.gov/WWW/BGH/sound.html) (from NASA Hypersonics Index). The extracted equations are sent to text2code service to convert the equation to equivalent python native and TensorFlow eager-compatible code. For example:

The equation extracted from text is as follows:

> a^2 = R * T * {1 + (gamma - 1) / ( 1 + (gamma-1) * [(theta/T)^2 * e^(theta/T) /(e^(theta/T) -1)^2]) }.

The response from text2code service for this text equation is as follows:

```css
a = tf.math.pow(R * T * (1 + (gamma-1 ) / ( 1 + ( gamma-1 ) *  ( tf.math.pow( theta/T,2) *  tf.math.exp( theta/T ) /  tf.math.pow(tf.math.exp( theta/T ) - 1,2 )))), 1/2)
```

K-CHAIN service builds a computational graph from this python Tensorflow-compatible code and then it is saved as a MetaGraph for further edits and inference later. The computational graph construction uses AutoGraph ([documentation](https://www.tensorflow.org/guide/autograph) and [research article](https://arxiv.org/abs/1810.08061)), which allows for codes that include conditional statements and loops too. In this demo, the response from text2code service is used to create a computational graph. 

Note that the service allows to provide _default values_ for certain input variables, so that they can be used during inference vene if they are not assigned by the user. Examples include value for gas constant of air _R = 286.0_ (in SI units).


In [4]:
inputPacket = {
                  "inputVariables": [
                    {
                        "name": "gamma",
                        "type": "double",
                        "value": "1.4"
                    },
                    {
                        "name": "R",
                        "type": "double",
                        "value": "286.0"
                    },
                    {
                        "name": "theta",
                        "type": "double",
                        "value": "3056.0"
                    },
                    {
                      "name": "T",
                      "type": "double"
                    }
                  ],
                  "outputVariables": [
                    {
                      "name": "a",
                      "type": "double"
                    }
                  ],
                   "equationModel" : "a = tf.math.pow(R * T *  (  1 + ( gamma-1 ) / ( 1 + ( gamma-1 ) *  ( tf.math.pow( theta/T,2) *  tf.math.exp( theta/T ) /  tf.math.pow( tf.math.exp( theta/T ) - 1,2 ))  ) ), 0.5)",
                   "modelName" : "speedOfSound"
                 }
r = requests.post(url_build, json=inputPacket)
r.json()

{'metagraphLocation': '../models/speedOfSound',
 'modelType': 'Physics',
 'trainedState': 0}

In [5]:
inputPacket = { 
   "modelName":"Turbo_getAir",
   "equationModel":"#  Utility to get the corrected airflow per area given the Mach number\n    fac2 = (gamma + 1.0) / (2.0 * (gamma - 1.0))\n    fac1 = tf.math.pow((1.0 + 0.5 * (gamma - 1.0) * mach * mach), fac2)\n    number = 0.50161 * tf.math.sqrt(gamma) * mach / fac1\n    getAir = (number)\n    ",
   "inputVariables":[ 
      { 
         "name":"mach",
         "type":"double"
      },
      { 
         "name":"gamma",
         "type":"double"
      }
   ],
   "outputVariables":[ 
      { 
         "name":"getAir",
         "type":"double"
      }
   ]
}
r = requests.post(url_build, json=inputPacket)
r.json()

{'metagraphLocation': '../models/Turbo_getAir',
 'modelType': 'Physics',
 'trainedState': 0}

The computational graph capturing this _SpeedOfSound_ model can be seen in __TensorBoard__. The model is as follows:
    <img src="figures for notebook/sos_tensorboard_pic.PNG" style="width: 80%">

Equation model contains python method

In [4]:
inputPacket = {
                  "inputVariables": [
                    {
                        "name": "gamma",
                        "type": "double",
                        "value": "1.4"
                    },
                    {
                        "name": "R",
                        "type": "double",
                        "value": "286.0"
                    },
                    {
                        "name": "theta",
                        "type": "double",
                        "value": "3056.0"
                    },
                    {
                      "name": "T",
                      "type": "double"
                    }
                  ],
                  "outputVariables": [
                    {
                      "name": "a",
                      "type": "double"
                    }
                  ],
                   "equationModel" : "def speedOfSound(gamma, R, theta, T):\
\n    a = tf.math.pow(R * T *  (  1 + ( gamma-1 ) / ( 1 + ( gamma-1 ) *  ( tf.math.pow( theta/T,2) *  tf.math.exp( theta/T ) /  tf.math.pow( tf.math.exp( theta/T ) - 1,2 ))  ) ), 0.5)\
\n    return a",
                   "modelName" : "speedOfSound"
                 }
r = requests.post(url_build, json=inputPacket)
r.json()

{'metagraphLocation': '../models/speedOfSound',
 'modelType': 'Physics',
 'trainedState': 0}

Equation model contains python method with nested call to another method

In [7]:
inputPacket = {
                  "inputVariables": [
                    {
                        "name": "Cp",
                        "type": "double",
                        "value": "140"
                    },
                    {
                        "name": "Cv",
                        "type": "double",
                        "value": "100"
                    },
                    {
                        "name": "R",
                        "type": "double",
                        "value": "286.0"
                    },
                    {
                        "name": "theta",
                        "type": "double",
                        "value": "3056.0"
                    },
                    {
                        "name": "T",
                        "type": "double"
                    }
                  ],
                  "outputVariables": [
                    {
                        "name": "a",
                        "type": "double"
                    },
                    {
                        "name": "gamma",
                        "type": "double"
                    }
                  ],
                   "equationModel" : \
'''def speedOfSound2(Cp, Cv, R, theta, T):
    gamma = getGamma(Cp, Cv)
    a = tf.math.pow(R * T *  (  1 + ( gamma-1 ) / ( 1 + ( gamma-1 ) *  ( tf.math.pow( theta/T,2) *  tf.math.exp( theta/T ) /  tf.math.pow( tf.math.exp( theta/T ) - 1,2 ))  ) ), 0.5)
    return a, gamma

def getGamma(Cp, Cv):
    return Cp/Cv
    ''',
                   "modelName" : "speedOfSound2"
                 }
r = requests.post(url_build, json=inputPacket)
r.json()

{'metagraphLocation': '../models/speedOfSound2',
 'modelType': 'Physics',
 'trainedState': 0}

Note that in case of such nested call, the output (gamma in this case) of any interim call (getGamma in this case) is not readily available to be queried. If that interim variable will either be set or requested by user then consider incorporating that method separately in the computational graph.

In [8]:
inputPacket = {
                  "inputVariables": [
                    {
                        "name": "TT",
                        "type": "double"
                    },
                    {
                        "name": "M",
                        "type": "double"
                    },
                    {
                        "name": "G",
                        "type": "double"
                    },
                    {
                        "name": "Q",
                        "type": "double"
                    }  
                  ],
                  "outputVariables": [
                    {
                      "name": "T",
                      "type": "double"
                    }
                  ],
                   "equationModel" : '''
def CAL_T(TT, M, G, Q):
    T = TT * TQTT(M,G)
    EPS = 0.00001
    
    Z = tf.math.pow(M, 2) - 2 * TT / 1.4 / T * (G / (G - 1) * (1 - T / TT) + Q / TT *(1 / (tf.math.exp(Q / TT) - 1) - 1 / (tf.math.exp(Q / T) - 1)))
    Z2 = Z
    T2 = T
    T = T2 + 100
    while abs(Z) > EPS:
        #Z = tf.math.pow(M, 2) - 2 * TT / CAL_GAM(T, G, Q) / T * (G / (G - 1) * (1 - T / TT) + Q / TT *(1 / (tf.math.exp(Q / TT) - 1) - 1 / (tf.math.exp(Q / T) - 1)))
        Z = tf.math.pow(M, 2) - 2 * TT / 1.4 / T * (G / (G - 1) * (1 - T / TT) + Q / TT *(1 / (tf.math.exp(Q / TT) - 1) - 1 / (tf.math.exp(Q / T) - 1)))
        T1 = T2
        Z1 = Z2
        T2 = T
        Z2 = Z
        T = T2 - Z2 * (T2 - T1) / (Z2 - Z1)
    if M <= .1: 
        T = TT * TQTT(M,G)
    return T

def TQTT(M, G):
    return tf.math.pow((1 + (G - 1) / 2 * tf.math.pow(M, 2)), -1)

''',
                   "modelName" : "CAL_T"
                 }
r = requests.post(url_build, json=inputPacket)
r.json()

{'metagraphLocation': '../models/CAL_T',
 'modelType': 'Physics',
 'trainedState': 0}

### Building models with experimental data 

**Note: This capability for data-driven models will be made available again in next release.**


If a dataset has the values recorded for input and output variables for a model, then even if a relationship of those variables has not yet been extracted by the ANSWER agent, a data-driven model can be created to capture the relationship and perform inference. In this demo, a neural network model relating the inputs to output is constructed and trained with the dataset. Internally, \__createNNModel()_ and _fitModel()_ methods are being used. Note that _fitModel()_ can also be used to update an existing model as more data becomes available for training.  


The MetaGraph of TensorFlow computational graph is stored in _models_ folder with name _ForceModel_. The model is of type Neural Network, so eventually ANSWER agent can look for information sources to convert to more exact form of knowledge with physics equations and then data is used for validation. The trainedState is 1 as data-driven model has been fitted to the training dataset. 

The computational graph capturing this _ForceModel_ can be seen in __TensorBoard__. The model is as follows:
    <img src="figures for notebook/forcenn_tensorboard_pic.PNG" style="width: 80%">

The depicted graph is a hierarchical model, where _NN_ can be expanded to show the computations as follows:
    <img src="figures for notebook/forcenn2_tensorboard_pic.PNG" style="width: 80%">



# Model evaluate demonstrations

### Evaluate a physics model where all relevant inputs are provided

In [9]:
evalPacket = {
  "inputVariables": [
    {
      "name": "T",
      "type": "double",
      "value": "[300.0, 270.0, 370.0, 300.0]"
    },
    {
      "name": "R",
      "type": "double",
      "value": "286.0"
    },
    {
      "name": "gamma",
      "type": "double",
      "value": "1.4"
    },
    {
      "name": "theta",
      "type": "double",
      "value": "3056.0"
    }
  ],
  "modelName": "speedOfSound",
  "outputVariables": [
    {
      "name": "a",
      "type": "double"
    }
  ]
}
r = requests.post(url_evaluate, json=evalPacket)
r.json()

{'error': '',
 'inputVariables': [{'name': 'T',
   'type': 'double',
   'value': '[300.0, 270.0, 370.0, 300.0]'},
  {'name': 'R', 'type': 'double', 'value': '286.0'},
  {'name': 'gamma', 'type': 'double', 'value': '1.4'},
  {'name': 'theta', 'type': 'double', 'value': '3056.0'}],
 'outputVariables': [{'name': 'a',
   'type': 'double',
   'value': '[346.50601552,328.7685916 ,384.51401102,346.50601552]'}]}

__Explaining ideal output:__

```
{'outputVariables': [{'name': 'a',
   'type': 'double',
   'value': '[346.50601552]'}]}
```
The provided input values were used with speedOfSound model built above to compute the output. 

### Evaluate a physics model where values for all inputs are not provided 

In [10]:
evalPacket = {
  "inputVariables": [
    {
      "name": "T",
      "type": "double",
      "value": "[273.00, 300.00, 350.00, 400.00]"
    }
  ],
  "modelName": "speedOfSound",
  "outputVariables": [
    {
      "name": "a",
      "type": "double"
    }
  ]
}
r = requests.post(url_evaluate, json=evalPacket)
r.json()

{'defaultsUsed': [{'name': 'theta', 'value': '3056.0'},
  {'name': 'gamma', 'value': '1.4'},
  {'name': 'R', 'value': '286.0'}],
 'error': '',
 'inputVariables': [{'name': 'T',
   'type': 'double',
   'value': '[273.00, 300.00, 350.00, 400.00]'}],
 'outputVariables': [{'name': 'a',
   'type': 'double',
   'value': '[330.58687602,346.50601552,374.09061791,399.56414241]'}]}

__Explaining ideal output:__
```
{'defaultsUsed': [{'name': 'R', 'value': '286.0'},
  {'name': 'theta', 'value': '3056.0'},
  {'name': 'gamma', 'value': '1.4'}],
 'outputVariables': [{'name': 'a',
   'type': 'double',
   'value': '[330.58687602]'}]}
```
Since computation of speed of sound needs the value of _R_, _gamma_, and _theta_ and those values were not provided during the query, it uses default values if they were provided during model build. If default values are used, then it informs the CurationManager and hence the user, so that assumption that default values are applicable in this computation are made explicit to the user.    


# Multiple output variables

In [11]:
evalPacket = {
  "inputVariables": [
    {
      "name": "T",
      "type": "double",
      "value": "[273.00, 300.00, 350.00, 400.00]"
    }
  ],
  "modelName": "speedOfSound2",
  "outputVariables": [
    {
      "name": "a",
      "type": "double"
    },
    {
        "name": "gamma",
        "type": "double"
    }
  ]
}
r = requests.post(url_evaluate, json=evalPacket)
r.json()

{'defaultsUsed': [{'name': 'Cp', 'value': '140'},
  {'name': 'Cv', 'value': '100'},
  {'name': 'R', 'value': '286.0'},
  {'name': 'theta', 'value': '3056.0'}],
 'error': '',
 'inputVariables': [{'name': 'T',
   'type': 'double',
   'value': '[273.00, 300.00, 350.00, 400.00]'}],
 'outputVariables': [{'name': 'a',
   'type': 'double',
   'value': '[330.58687602,346.50601552,374.09061791,399.56414241]'},
  {'name': 'gamma', 'type': 'double', 'value': '[1.4,1.4,1.4,1.4]'}]}

# Inverse queries where model output values are given and input range is given

In [12]:
evalPacket = {
  "inputVariables": [
    {
        "name": "T",
        "type": "double",
        "minValue": "100.0",
        "maxValue": "600.0"
    }
  ],
  "modelName": "speedOfSound",
  "outputVariables": [
    {
        "name": "a",
        "type": "double",
        "value": "350.0"
    }
  ]
}
r = requests.post(url_evaluate, json=evalPacket)
r.json()

{'defaultsUsed': [{'name': 'theta', 'value': '3056.0'},
  {'name': 'gamma', 'value': '1.4'},
  {'name': 'R', 'value': '286.0'}],
 'error': '0.018126639999991312',
 'inputVariables': [{'maxValue': '600.0',
   'minValue': '100.0',
   'name': 'T',
   'type': 'double',
   'value': '306.13654863461096'}],
 'outputVariables': [{'name': 'a',
   'type': 'double',
   'value': '[350.01812664]'}]}

In [13]:
evalPacket = {
  "inputVariables": [
    {
        "name": "T",
        "type": "double",
        "minValue": "100.0",
        "maxValue": "600.0"
    }
  ],
  "modelName": "speedOfSound",
  "outputVariables": [
    {
        "name": "a",
        "type": "double",
        "value": "350.0"
    },
    {
        "name": "gamma",
        "type": "double",
        "value": "1.4"
    }
  ]
}
r = requests.post(url_evaluate, json=evalPacket)
r.json()

{'defaultsUsed': [{'name': 'gamma', 'value': '1.4'},
  {'name': 'R', 'value': '286.0'},
  {'name': 'theta', 'value': '3056.0'}],
 'error': '0.0035821499999997286',
 'inputVariables': [{'maxValue': '600.0',
   'minValue': '100.0',
   'name': 'T',
   'type': 'double',
   'value': '306.0984158543412'}],
 'outputVariables': [{'name': 'a',
   'type': 'double',
   'value': '[349.99641785]'},
  {'name': 'gamma', 'type': 'double', 'value': '[1.4]'}]}

### Validating Input Packets for Correctness

While some inputs are validated within the codes in the service, the input packet is validated by Swagger at the REST endpoint before calling method from the package. If _value_ or _name_ field of an input variable is missing or if the field _inputVariables_, _outputVariables_, or _modelName_ is missing, then it leads to ambiguity and key information to conduct inference is missing. Thus, a validationError is caught by the packet validation, which checks for all required entries.   

In [14]:
evalPacket = {
  "inputVariables": [
    {
      "name": "T",
      "type": "double",
      "value": "300.00"
    },
    {
      "name": "R",
      "type": "double"
    }
  ],
  "modelName": "speedOfSound",
  "outputVariables": [
    {
      "name": "a",
      "type": "double"
    }
  ]
}
r = requests.post(url_evaluate, json=evalPacket)
r.json()

{'detail': 'The server encountered an internal error and was unable to complete your request. Either the server is overloaded or there is an error in the application.',
 'status': 500,
 'title': 'Internal Server Error',
 'type': 'about:blank'}

Try replacing current entry of:
```
    {
      "name": "R",
      "type": "double"
    }
```
with additional field of _value_: 
```
    {
      "name": "R",
      "type": "double",
      "value": 276.0
    }
```
However, then we should get following response as _value_ is expected to be a string:
```
{'detail': "276.0 is not of type 'string'",
 'status': 400,
 'title': 'Bad Request',
 'type': 'about:blank'}
```

Finally, if you replace the entry with:
```
    {
      "name": "R",
      "type": "double",
      "value": "276.0"
    }
```
We should get the desired response:
```
{'defaultsUsed': [{'name': 'gamma', 'value': '1.4'},
  {'name': 'theta', 'value': '3056.0'}],
 'outputVariables': [{'name': 'a',
   'type': 'double',
   'value': '[340.39431879]'}]}
```
where, default values of _gamma_ and _theta_ were used in the computation with the provided value of _R_ (276.0) in lieu of the default value 286.0

### Try to evaluate a physics model with no input values provided

In [15]:
evalPacket = {
  "inputVariables": [
  ],
  "modelName": "speedOfSound",
  "outputVariables": [
    {
      "name": "a",
      "type": "double"
    }
  ]
}
r = requests.post(url_evaluate, json=evalPacket)
r.json()

{'defaultsUsed': [{'name': 'gamma', 'value': '1.4'},
  {'name': 'R', 'value': '286.0'}],
 'error': '',
 'inputVariables': [],
 'missingVar': 'T',
 'outputVariables': [{'name': 'a', 'type': 'double', 'value': None}]}

__Explanation for ideal output:__
```
{'missingVar': 'T',
 'outputVariables': [{'name': 'a', 'type': 'double', 'value': None}]}
 ```
 
 Here the computation for output _a_ (speed of sound) cannot proceed without the value of _T_ (temperature of gas) as a default value has not been provided for this input. Thus, the service returns that variable _T_ is missing and output of current computation is None.

### Evaluate with a data-driven model where all relevant inputs are provided

**Note: This capability with data-driven models will be made available again in next release.**

For _Mass = 2.0_ and _Acceleration = 0.1_, a simple data-driven model trained with 100 examples when evaluated with the model-agnostic evaluate service gives the following output:
```
{'outputVariables': [{'name': 'Force',
   'type': 'double',
   'value': '[0.206]'}]}
```
Here, the output Force of type double is estimated to be 0.206. The estimate might be slightly different based on model training.

However, if _inputVariables_ are assigned values away from the training set, such as, _Mass = 0.5_ and _Acceleration = 0.1_, then the output is incorrect by an order of magnitude. For example:
```
{'outputVariables': [{'name': 'Force',
   'type': 'double',
   'value': '[0.627]'}]}
```
The estimate might be different based on model training.

Thus, in future for each data-driven model fitted to a training dataset, we will also characterize the region of trust of that model and alert the user if a query tries to exercise the model beyond its region of competence based on training data, model structure, and output uncertainty. 

## Model append demonstrations

### Append physics models with simple equations provided as strings to existing models:

### Example 1. Append to Newtons2LawModel

Case: Input nodes are shared with new model fragment

In [16]:
inputPacket = {
                  "inputVariables": [
                    {
                      "name": "Mass",
                      "type": "double",
                      "value": "1.0"
                    },
                    {
                      "name": "Velocity",
                      "type": "double"
                    }
                  ],
                  "outputVariables": [
                    {
                      "name": "Momentum",
                      "type": "double"
                    }
                  ],
                   "equationModel" : "Momentum = Mass * Velocity",
                   "modelName" : "Momentum",
                   "targetModelName" : "Newtons2LawModel"
                 }
#send request to append model
r = requests.post(url_append, json=inputPacket)

#see the response
r.json()

{'metagraphLocation': '../models/Newtons2LawModel', 'modelType': 'Physics'}

The computational graph capturing initial _Newtons2Law_ model can be seen in __TensorBoard__. The model is as follows:
<p><img src="figures for notebook/n2l_kchain_1.PNG" style="width: 20%">

The computational graph capturing updated _Newtons2Law_ model can be seen in __TensorBoard__. Note that the input node is reused and shared with the appended model fragment. The model is as follows:
<p><img src="figures for notebook/n2l_kchain_2.PNG" style="width: 40%">

### Example 2. Append to speedOfSound Model

<p>Case 1: Output nodes are used as inputs with new model fragment
<p>Case 2: Input nodes are outputs of new model fragment

In [17]:
inputPacket = {
                "inputVariables": [
                    {
                        "name": "a",
                        "type": "float"
                    },
                    {
                        "name": "objVelocity",
                        "type": "float"
                    }
                ],
                "outputVariables": [
                    {
                      "name": "machNumber",
                      "type": "float"
                    }
                ],
                "equationModel" : "machNumber = objVelocity/a",
                "modelName" : "machNumber",
                "targetModelName" : "speedOfSound"
             }

#send request to append model
r = requests.post(url_append, json=inputPacket)

#see the response
print(r.json())

{'metagraphLocation': '../models/speedOfSound', 'modelType': 'Physics'}


The computational graph capturing initial _speedOfSound_ model can be seen in __TensorBoard__. The model is as follows:
<p><img src="figures for notebook/sos_kchain_2.PNG" style="width: 30%">

In [18]:
inputPacket = {
                "inputVariables": [
                    {
                        "name": "Cp",
                        "type": "float",
                        "value": "1.4"
                    },
                    {
                        "name": "Cv",
                        "type": "float",
                        "value":"1.0"
                    }
                ],
                "outputVariables": [
                    {
                      "name": "gamma",
                      "type": "float"
                    }
                ],
                "equationModel" : "gamma = Cp/Cv",
                "modelName" : "gamma",
                "targetModelName" : "speedOfSound"
             }

#send request to append model
r = requests.post(url_append, json=inputPacket)

#see the response
print(r.json())

{'metagraphLocation': '../models/speedOfSound', 'modelType': 'Physics'}


The computational graph capturing initial _speedOfSound_ model can be seen in __TensorBoard__. The model is as follows:
<p><img src="figures for notebook/sos_kchain_3.PNG" style="width: 30%">

In [19]:
evalPacket = {
  "inputVariables": [
    {
      "name": "T",
      "type": "double",
      "value": "[273.00, 300.00, 350.00, 400.00]"
    }
  ],
  "modelName": "speedOfSound",
  "outputVariables": [
    {
      "name": "a",
      "type": "double"
    },
    {
        "name": "gamma",
        "type": "double"
    }
  ]
}
r = requests.post(url_evaluate, json=evalPacket)
r.json()

{'defaultsUsed': [{'name': 'Cp', 'value': '1.4'},
  {'name': 'Cv', 'value': '1.0'},
  {'name': 'theta', 'value': '3056.0'},
  {'name': 'R', 'value': '286.0'}],
 'error': '',
 'inputVariables': [{'name': 'T',
   'type': 'double',
   'value': '[273.00, 300.00, 350.00, 400.00]'}],
 'outputVariables': [{'name': 'a',
   'type': 'double',
   'value': '[330.58687602,346.50601552,374.09061791,399.56414241]'},
  {'name': 'gamma', 'type': 'double', 'value': '[1.4,1.4,1.4,1.4]'}]}

In [20]:
import numpy as np

T = np.linspace(100.0,1000.0, num=100)
evalPacket = {
  "inputVariables": [
    {
      "name": "T",
      "type": "double",
      "value": np.array2string(T, precision=3, separator=',')
    }
  ],
  "modelName": "speedOfSound",
  "outputVariables": [
    {
      "name": "a",
      "type": "double"
    },
    {
        "name": "gamma",
        "type": "double"
    }
  ]
}
r = requests.post(url_evaluate, json=evalPacket)
rj = r.json()
a = np.fromstring(rj["outputVariables"][0]["value"][1:-1], dtype=float, sep=',')
print(a)
gamma = np.fromstring(rj["outputVariables"][1]["value"][1:-1], dtype=float, sep=',')
print(gamma)

[200.09997501 208.99769472 217.53177418 225.74345854 233.66673923
 241.33002257 248.75652927 255.96842415 262.98258414 269.81439787
 276.47732927 282.98322564 289.34256199 295.56463755 301.6577358
 307.62925691 313.48582882 319.2327754  324.87671496 330.42183907
 335.8725155  341.23270591 346.50601552 351.69573752 356.80489273
 361.83626493 366.79243231 371.67579524 376.4880751  381.23244545
 385.91037446 390.52376592 395.07443955 399.56414241 403.99455844
 408.36731632 412.68399578 416.94613276 421.15522338 425.31227243
 429.41961929 433.47819588 437.48936252 441.45444861 445.37475298
 449.25154409 453.08606009 456.87950871 460.63306715 464.34788191
 468.0246662  471.66531364 475.27047147 478.84116246 482.37837852
 485.88308081 489.35619999 492.79863651 496.21126099 499.59491468
 502.95040993 506.27816612 509.57967165 512.85528812 516.10571831
 519.33163927 522.5337031  525.71253764 528.86874726 532.00291363
 535.11559645 538.20733427 541.27830846 544.32969311 547.36162878
 550.374576

In [21]:
evalPacket = {
  "inputVariables": [
    {
      "name": "T",
      "type": "double",
      "value": "[273.00, 300.00, 350.00, 400.00]"
    }
  ],
  "modelName": "speedOfSound",
  "outputVariables": [
    {
      "name": "a",
      "type": "double"
    },
    {
        "name": "gamma",
        "type": "double"
    },
    {
        "name":"machNumber",
        "type": "double"
    }
  ]
}
r = requests.post(url_evaluate, json=evalPacket)
r.json()

{'defaultsUsed': [{'name': 'R', 'value': '286.0'},
  {'name': 'Cp', 'value': '1.4'},
  {'name': 'Cv', 'value': '1.0'}],
 'error': '',
 'inputVariables': [{'name': 'T',
   'type': 'double',
   'value': '[273.00, 300.00, 350.00, 400.00]'}],
 'missingVar': 'objVelocity',
 'outputVariables': [{'name': 'a', 'type': 'double', 'value': None},
  {'name': 'gamma', 'type': 'double', 'value': None},
  {'name': 'machNumber', 'type': 'double', 'value': None}]}

In [22]:
evalPacket = {
  "inputVariables": [
    {
      "name": "T",
      "type": "double",
      "value": "[273.00, 300.00, 350.00, 400.00]"
    },
    {
        "name": "objVelocity",
        "type":"double",
        "value":"300.00" #[300.0, 300.0, 600.0, 600]
    }
  ],
  "modelName": "speedOfSound",
  "outputVariables": [
    {
      "name": "a",
      "type": "double"
    },
    {
        "name": "gamma",
        "type": "double"
    },
    {
        "name":"machNumber",
        "type": "double"
    }
  ]
}
r = requests.post(url_evaluate, json=evalPacket)
r.json()

{'defaultsUsed': [{'name': 'theta', 'value': '3056.0'},
  {'name': 'Cp', 'value': '1.4'},
  {'name': 'Cv', 'value': '1.0'},
  {'name': 'R', 'value': '286.0'}],
 'error': '',
 'inputVariables': [{'name': 'T',
   'type': 'double',
   'value': '[273.00, 300.00, 350.00, 400.00]'},
  {'name': 'objVelocity', 'type': 'double', 'value': '300.00'}],
 'outputVariables': [{'name': 'a',
   'type': 'double',
   'value': '[330.58687602,346.50601552,374.09061791,399.56414241]'},
  {'name': 'gamma', 'type': 'double', 'value': '[1.4,1.4,1.4,1.4]'},
  {'name': 'machNumber',
   'type': 'double',
   'value': '[0.90747704,0.86578584,0.80194473,0.75081812]'}]}

### Build, append, and evaluate with a gradient operation

In [23]:
inputPacket = {
                  "inputVariables": [
                    {
                        "name": "Time",
                        "type": "double"
                    }
                  ],
                  "outputVariables": [
                    {
                      "name": "Velocity",
                      "type": "double"
                    }
                  ],
                   "equationModel" : "Velocity = 10.0 + (2.1*Time)", 
                   "modelName" : "Velocity"
                 }
#send request to append model
r = requests.post(url_append, json=inputPacket)

#see the response
r.json()

{'metagraphLocation': '../models/Velocity', 'modelType': 'Physics'}

The computational graph capturing this _Velocity_ model can be seen in __TensorBoard__. The model is as follows:
<p><img src="figures for notebook/vel_kchain_1.PNG" style="width: 20%">

The depicted graph is a hierarchical model, where _Velocity_ node can be expanded to show the computations as follows:
<p><img src="figures for notebook/vel_kchain_2.PNG" style="width: 40%">

In [24]:
inputPacket = {
                  "inputVariables": [
                    {
                      "name": "Velocity",
                      "type": "double"
                    },
                    {
                        "name": "Time",
                        "type": "double"
                    }
                  ],
                  "outputVariables": [
                    {
                      "name": "Acc",
                      "type": "double"
                    }
                  ],
                   "equationModel" : "Acc = tf.gradients(Velocity, Time, stop_gradients = [Time])[0]",
                   "modelName" : "Acc",
                   "targetModelName" : "Velocity"
                 }
#send request to build model
r = requests.post(url_append, json=inputPacket)

#see the response
r.json()

{'metagraphLocation': '../models/Velocity', 'modelType': 'Physics'}

The computational graph capturing this appended _Velocity_ model can be seen in __TensorBoard__. The model is as follows:
<p><img src="figures for notebook/acc_kchain_1.PNG" style="width: 20%">

The depicted graph is a hierarchical model, where _Acc_ node can be expanded to show the computations as follows:
<p><img src="figures for notebook/acc_kchain_2.PNG" style="width: 30%">

In [25]:
evalPacket = {
  "inputVariables": [
    {
      "name": "Time",
      "type": "double",
      "value": "[1.0]"
    }
  ],
  "modelName": "Velocity",
  "outputVariables": [
    {
      "name": "Acc",
      "type": "double"
    }
  ]
}
r = requests.post(url_evaluate, json=evalPacket)
r.json()

{'error': '',
 'inputVariables': [{'name': 'Time', 'type': 'double', 'value': '[1.0]'}],
 'outputVariables': [{'name': 'Acc', 'type': 'double', 'value': '[2.1]'}]}

## The End